# Karabük 2025 Orman Yangınları Analizi

--- 

## Proje Hakkında
Bu çalışma, **2025 yazında Karabük ilinde** meydana gelen orman yangınlarının çevresel etkilerini tespit etmek, izlemek ve nicel olarak analiz etmek amacıyla hazırlanmıştır. Google Earth Engine (GEE) bulut platformunun hesaplama gücü ve Sentinel-2 uydusunun yüksek çözünürlüklü görüntüleri kullanılarak, yangın öncesi ve sonrası bitki örtüsü değişimleri incelenmiştir.

### Hedefler
1.  **İl Geneli Tarama:** Karabük il sınırları içindeki tüm potansiyel yanmış alanların hızlı tespiti ve görsel doğrulaması.
2.  **Detaylı Hasar Analizi:** Belirlenen 6 kritik yangın bölgesinde (Aladağlar, Ovacık, vb.) yangın şiddetinin (Burn Severity) sınıflandırılması.
3.  **Etki Raporlama:** Her bölge için hasar seviyesi (yüksek/orta) istatistiklerinin üretilmesi ve haritalanması.

---

## Analiz Metodolojisi

Analiz, uzaktan algılama literatüründe kabul görmüş standart indekslere dayanmaktadır:

### 1. Spektral İndeksler
*   **NDVI (Normalized Difference Vegetation Index):** Bitki sağlığını ve yoğunluğunu ölçer. Yangın sonrası düşüş gösterir.
    *   `NDVI = (NIR - RED) / (NIR + RED)`
*   **NBR (Normalized Burn Ratio):** Yanmış alanları tespit etmek için özel olarak geliştirilmiştir. Sağlıklı bitki örtüsü ile yanmış toprağı ayırt eder.
    *   `NBR = (NIR - SWIR) / (NIR + SWIR)`

### 2. Değişim Analizi (Change Detection)
Yangın öncesi ve sonrası görüntüler arasındaki fark (Delta) hesaplanarak hasar belirlenir:
*   **dNBR (Delta NBR):** `Öncesi NBR - Sonrası NBR`. Değer ne kadar yüksekse yanma o kadar şiddetlidir.
*   **dNDVI (Delta NDVI):** `Öncesi NDVI - Sonrası NDVI`. Vejetasyon kaybını gösterir.

### 3. Yanma Şiddeti Sınıflandırması (USGS Standartları)
dNBR değerlerine göre alanlar şu sınıflara ayrılır:
*   **Yanmamış:** < 0.1
*   **Düşük Şiddet:** 0.1 - 0.27
*   **Orta-Düşük Şiddet:** 0.27 - 0.44
*   **Orta-Yüksek Şiddet:** 0.44 - 0.66
*   **Yüksek Şiddet:** > 0.66

In [ ]:
# Gerekli kütüphanelerin ve modüllerin yüklenmesi
import os
import json
import ee
import pandas as pd
from IPython.display import IFrame, display, HTML

# Proje modülleri (gee klasörü altındaki yardımcı fonksiyonlar)
from gee.pipeline import run_pipeline
from gee.utils import ee_init
from gee.aoi import get_aoi

# Tablo Görünüm Ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Kütüphaneler başarıyla yüklendi.")

In [ ]:
# Google Earth Engine Başlatma
ee_project = 'karabuk-orman-yangini-2025'

try:
    ee_init(project=ee_project)
    print('Google Earth Engine bağlantısı kuruldu.')
except Exception as e:
    print(f'Başlatma hatası: {e}')
    print('Lütfen terminalde "earthengine authenticate" komutunu çalıştırarak yetki verin.')

## 1. Analiz Parametreleri

Yangın öncesi ve sonrası dönemler, Karabük bölgesindeki mevsimsel değişimler ve bulutluluk oranları dikkate alınarak **geniş bir aralıkta** belirlenmiştir. Bu sayede bulutsuz ve kaliteli görüntü (kompozit) oluşturma şansı artırılmıştır.

*   **Referans Dönem:** Haziran - Temmuz 2025 (Yangın öncesi sağlıklı bitki örtüsü)
*   **Hasar Tespit Dönemi:** Eylül - Ekim 2025 (Yangınların söndüğü ve dumanın dağıldığı dönem)

In [ ]:
# Analiz Tarihleri
pre_start = '2025-06-10'
pre_end   = '2025-06-30'

post_start = '2025-09-05'
post_end   = '2025-09-30'

# Dosya Yolları
base_out_dir = 'results'

print(f"Analiz Takvimi:\n  • Öncesi:  {pre_start} — {pre_end}\n  • Sonrası: {post_start} — {post_end}")

In [ ]:
def show_html(path, height=600):
    """HTML harita dosyasını notebook içinde görüntülemek için yardımcı fonksiyon."""
    if not os.path.exists(path):
        display(HTML(f"<div style='padding:10px; background-color:#ffebee; color:#c62828; border-left:5px solid #c62828;'><b>Dosya bulunamadı:</b> {path}</div>"))
        return
    print(f"Harita görüntüleniyor: {os.path.basename(path)}")
    display(IFrame(src=path, width='100%', height=height))

## 2. İl Geneli Genel Tarama

Bu aşamada tüm Karabük ili için **dNBR (Yanma Fark İndeksi)** ve **dNDVI (Bitki Örtüsü Farkı)** haritaları oluşturulur. 
*   **Amaç:** Büyük yangın alanlarının yerini doğrulamak ve genel hasar durumunu görmek.
*   **Yöntem:** USGS Standartlarında dNBR sınıflandırması (Severity) uygulanır. Bellek güvenliği için 100m ölçeklendirme kullanılır.

In [ ]:
print("--- 1. AŞAMA: Tüm Karabük İl Geneli Analiz ---")

province_out_dir = os.path.join(base_out_dir, "il_geneli")
os.makedirs(province_out_dir, exist_ok=True)

province_bbox_path = "karabuk_province_bbox.geojson"

try:
    # 1. İl sınırlarını al ve kareye çevir (Bounding Box Optimazasyonu)
    full_province_geom = get_aoi("KARABUK_PROVINCE")
    province_bbox = full_province_geom.bounds()
    
    # Geçici analiz sınır dosyasını kaydet
    feature = {"type": "Feature", "properties": {"name": "KARABUK_BBOX"}, "geometry": province_bbox.getInfo()}
    with open(province_bbox_path, "w", encoding="utf-8") as f:
        json.dump(feature, f)

    # 2. Pipeline Çalıştır (Severity AÇIK, Scale=100)
    province_outputs = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=province_bbox_path,
        out_dir=province_out_dir,
        area_scale=100,      # Geniş alan taraması için 100m ölçek (Maksimum bellek güvenliği)
        min_patch_ha=2,   
        project=ee_project,
        skip_severity=False,  # İl genelinde de severity analizi yap (USGS Standardı)
        overlay_boundary=full_province_geom # İl sınırlarını görselde çizmek için
    )

    print(f"İl analizi tamamlandı.\nÇıktılar: {province_out_dir}")

    # Dönen istatistikleri göster
    if os.path.exists(province_outputs.get("summary_csv", "")):
        display(pd.read_csv(province_outputs["summary_csv"]))

except Exception as e:
    print(f"İl genel analizi sırasında hata: {e}")
finally:
    if os.path.exists(province_bbox_path): os.remove(province_bbox_path)

### İl Geneli Değişim Haritaları (dNBR ve dNDVI)
Aşağıdaki haritalar yanma izlerini gösterir. 
*   **dNBR:** Yanmış alanları gösteren birincil indeks.
*   **dNDVI:** Vejetasyon kaybını gösteren yardımcı indeks.

In [ ]:
if 'province_outputs' in locals():
    if 'severity_map' in province_outputs:
        print("Karabük İli Severity (USGS) Haritası:")
        show_html(province_outputs['severity_map'])
    
    if 'dnbr_map' in province_outputs:
        print("Karabük İli dNBR Haritası:")
        show_html(province_outputs['dnbr_map'])

## 3. Odaklanmış Bölgesel Analizler

Bu aşamada 6 farklı kritik bölge için **10km x 10km** karelik alanlarda derinlemesine analiz yapılır. Bu analizde:
1.  **Severity (Şiddet) Haritası** üretilir.
2.  Yüksek çözünürlüklü rapor görselleri oluşturulur.

In [ ]:
# Analiz Bölgeleri Tanımları (Geniş Tarih Aralığı ile)
FIRE_ZONES = [
    {
        "name": "1_Aladaglar", "center": [32.5997, 41.1225],
        "pre_start": "2025-06-15", "pre_end": "2025-07-20",
        "post_start": "2025-09-01", "post_end": "2025-10-15"
    },
    {
        "name": "2_Buyuk_Ovacik_Kisla", "center": [32.9890, 41.0341],
        "pre_start": "2025-06-01", "pre_end": "2025-07-20",
        "post_start": "2025-09-01", "post_end": "2025-10-20"
    },
    {
        "name": "3_Cumayani", "center": [32.6688, 41.1036],
        "pre_start": "2025-06-01", "pre_end": "2025-07-20",
        "post_start": "2025-09-01", "post_end": "2025-10-20"
    },
    {
        "name": "4_Cavuslar", "center": [32.7914, 41.1638],
        "pre_start": "2025-06-01", "pre_end": "2025-07-20",
        "post_start": "2025-09-01", "post_end": "2025-10-20"
    },
    {
        "name": "5_Hamzalar_Aricak", "center": [32.5891, 41.1985],
        "pre_start": "2025-06-01", "pre_end": "2025-07-20",
        "post_start": "2025-09-01", "post_end": "2025-10-20"
    },
    {
        "name": "6_Eflani", "center": [32.9587, 41.4231],
        "pre_start": "2025-06-01", "pre_end": "2025-07-20",
        "post_start": "2025-09-01", "post_end": "2025-10-20"
    }
]

In [ ]:
print("--- 2. AŞAMA: Bölgesel Detaylı Analizler ---")

zone_results = {}

for zone in FIRE_ZONES:
    name = zone['name']
    display(HTML(f"<h3>Analiz Alanı: {name.replace('_', ' ')}</h3>"))
    
    zone_dir = os.path.join(base_out_dir, "yanginlar", name)
    os.makedirs(zone_dir, exist_ok=True)
    
    center = ee.Geometry.Point(zone['center'])
    zone_geom = center.buffer(5000).bounds()
    
    temp_geojson = f"results/temp_{name}.geojson"
    if os.path.exists(temp_geojson): os.remove(temp_geojson)

    feature = {"type": "Feature", "properties": {"name": name}, "geometry": zone_geom.getInfo()}
    with open(temp_geojson, "w", encoding="utf-8") as f:
        json.dump(feature, f)
        
    try:
        # Analizi Başlat (Severity AKTİF)
        outputs = run_pipeline(
            pre_start=zone['pre_start'], pre_end=zone['pre_end'],
            post_start=zone['post_start'], post_end=zone['post_end'],
            aoi_geojson=temp_geojson,
            out_dir=zone_dir,
            area_scale=10,
            min_patch_ha=0.1,
            project=ee_project,
            skip_severity=False
        )
        zone_results[name] = outputs
        print(f"{name} analizi tamamlandı.")
        
        if 'summary_csv' in outputs:
            df = pd.read_csv(outputs['summary_csv'])
            important_metrics = ['severity_4_yuksek', 'severity_3_orta_yuksek']
            display(df[df['metric'].isin(important_metrics)])

    except Exception as e:
        print(f"HATA: {name} analizi başarısız. {e}")
    finally:
        if os.path.exists(temp_geojson): os.remove(temp_geojson)

print("\nTüm bölgesel analizler tamamlandı.")

## 4. Sonuç Özeti ve Karşılaştırma
Aşağıdaki tablo, analiz edilen bölgelerdeki **yüksek ve orta-yüksek şiddetli** alanları özetlemektedir.

In [ ]:
summary_data = []

for zone_name, res in zone_results.items():
    csv_path = res.get('summary_csv')
    if csv_path and os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        metrics = dict(zip(df['metric'], df['value']))
        
        summary_data.append({
            "Bölge": zone_name,
            "Yüksek Şiddet (Ha)": metrics.get('severity_4_yuksek', 0),
            "Orta-Yüksek Şiddet (Ha)": metrics.get('severity_3_orta_yuksek', 0)
        })

if summary_data:
    df_summary = pd.DataFrame(summary_data)
    df_summary = df_summary.sort_values(by="Yüksek Şiddet (Ha)", ascending=False)
    display(HTML("<h4>Bölgesel Hasar Özeti</h4>"))
    display(df_summary.style.background_gradient(cmap='Reds', subset=['Yüksek Şiddet (Ha)']))
else:
    print("Henüz sonuç üretilmedi.")